In [1]:
import xarray as xr
from cmip6_preprocessing.preprocessing import combined_preprocessing
import PyCO2SYS as pyco2 # i have to install this package through terminal for it to work
import intake
import matplotlib.pyplot as plt
import numpy as np
import warnings
import gsw
warnings.filterwarnings("ignore")

In [2]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

In [5]:
cat = col.search(source_id = ['CanESM5-CanOE', 'CanESM5', 'ACCESS-ESM1-5',
                        'MPI-ESM-1-2-HAM', 'IPSL-CM6A-LR', 'MIROC-ES2L',
                        'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'NorCPM1',
                        'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM', 'GFDL-ESM4'], variable_id= ['so', 'thetao', 'dissic', 'talk'], experiment_id= 'piControl', table_id = 'Omon', grid_label = 'gn')
dd = cat.to_dataset_dict(
    zarr_kwargs={'consolidated': True, 'use_cftime':True},
    storage_options={'token': 'anon'},
    preprocess=combined_preprocessing,
    aggregate=False)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


MIROC-ES2L: No units found
CESM2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
CESM2-WACCM: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
MIROC-ES2L: No units found
MIROC-ES2L: No units found
CESM2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
MIROC-ES2L: No units found
CESM2-WACCM: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
CESM2-WACCM: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
CESM2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
M

This is copied from Graeme's code https://github.com/gmacgilchrist/cmip6/blob/74522b1593a3cac6174ec4e4ddcf6645204a7a86/dev_cmip6.ipynb

It needs some changes-`temperature_in`, `temperature_out`, `pressure_in`, and `pressure_out`.

Presumably 1035 is a conversion for something but TBD the exact nature of the conversion...I gotta do some math

In [ ]:
def calc_PpCO2(ds):
    results = pyco2.sys(par1=ds['talk']*1035.0,par2=ds['dissic']*1035.0,par1_type=1,par2_type=2,pressure=0)
    return ds['talk'].copy(data=results['pCO2'])

`results = pyco2.sys(par1=None, par2=None, par1_type=None, par2_type=None, **kwargs)`

### Arguments

`par1` and `par2`: values of two different carbonate system parameters.

`par1_type` and `par2_type`: which types of parameter par1 and par2 are.


Total alkalinity (type `1`) in μmol·kg−1.
> <font color='red'>alkalinity in CMIP6 is in mol/m3 </font> 

Dissolved inorganic carbon (type `2`) in μmol·kg−1.
> <font color='red'>DIC in CMIP6 is in mol/m3 </font> 

pH (type `3`) on the Total, Seawater, Free or NBS scale1. Which scale is given by the argument opt_pH_scale.

Partial pressure of CO2 (type `4`) in μatm, Fugacity of CO2 (type `5`) in μatm, Aqueous CO2 (type `8`) in μmol·kg−1, or Dry mole fraction of CO2 (type `9`) in ppm.

Carbonate ion (type `6`) in μmol·kg−1.

Bicarbonate ion (type `7`) in μmol·kg−1.

### Kwargs/hydrographic conditions

`salinity`: practical salinity (dimensionless).

`temperature`: temperature at which `par1` and `par2` arguments are provided in °C (default 25 °C).
> <font color='red'>in situ temp </font> 

`pressure`: water pressure at which `par1` and `par2` arguments are provided in dbar (default 0 dbar).
> pressure_in will be related to depth

> `gsw.p_from_z(z, lat, geo_strf_dyn_height=0, sea_surface_geopotential=0)` = dbar

> > Parameters

> > > zarray-like: Depth, positive up, m

> > > latarray-like: Latitude, -90 to 90 degrees

> > > geo_strf_dyn_heightarray-like

> > > > dynamic height anomaly, m^2/s^2

> > > > Note that the reference pressure, p_ref, of geo_strf_dyn_height must be zero (0) dbar.

> > > sea_surface_geopotentialarray-like

> > > > geopotential at zero sea pressure, m^2/s^2

> > Returns

> > > parray-like, dbar

> > > > sea pressure ( i.e. absolute pressure - 10.1325 dbar )


`temperature_out`: temperature at which results will be calculated in °C ("output conditions").
> <font color='red'>temperature_out = thetao (potential temp) </font> 

`pressure_out`: water pressure at which results will be calculated in dbar ("output conditions").
> <font color='red'>pressure out should be 0 since raised to surface </font> 

### Results (non-exhaustive but some that looked relevant)

`"dic"`: dissolved inorganic carbon in μmol·kg−1.

`"aqueous_CO2"/"aqueous_CO2_out"`: aqueous CO2 at input/output conditions in μmol·kg−1.

`"pCO2"/"pCO2_out"`: seawater partial pressure of CO2 at input/output conditions in μatm.

`"alkalinity"`: total alkalinity in μmol·kg−1.

`"pH"/"pH_out"`: pH at input/output conditions on the scale specified by input opt_pH_scale.

`"pH_total"/"pH_total_out"`: pH at input/output conditions on the Total scale.